In [ ]:
import pandas as pd
from os import walk
import os
import nltk
import pandas as pd
from nltk.probability import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords
from utils import * 
import os
import numpy as np
import pandas as pd
import math
import unicodedata

In [ ]:
sessions = np.arange(25, 76)
data=[]

for session in sessions:
    directory = "./Data/TXT/Session "+str(session)+" - "+str(1945+session)
    for filename in os.listdir(directory):
        f = open(os.path.join(directory, filename), encoding="utf8")
        if filename[0]==".": #ignore hidden files
            continue
        splt = filename.split("_")
        data.append([session, 1945+session, splt[0], f.read()])

        
df_speech = pd.DataFrame(data, columns=['Session','Year','ISO-alpha3 Code','Speech'])

df_speech.tail()

In [ ]:
df_speech.to_pickle('./Data/Throughput/Speeches_original.pickle')

In [ ]:
preprocessing_steps = ['prep_lower', 
                             'prep_tokenizewords', 
                             'prep_drop_sentenceindicators', 
                             'prep_drop_numbers', 
                             'prep_drop_stopwords',
                       'prep_listtosentence'
                      ]


In [ ]:
#df_speech = df_speech.set_index(['Session', 'Year', 'ISO-alpha3 Code'])
df_speech['speech'] = preprocess(df_speech['Speech'], preprocessing_steps)

In [ ]:
df_speech.to_pickle('./Data/Throughput/Speeches_preprocessed.pickle')

In [ ]:
df_happy = pd.read_excel('./Data/DataForTable2.1.xls')
df_happy = df_happy.rename({'year': 'Year'}, axis=1)
df_mapping = pd.read_csv('./Data/mapping_countries.csv')[['Country name', 'ISO-alpha3 Code']]

In [87]:
pd_final = pd.merge(pd.merge(df_happy, df_mapping, on='Country name'), df_speech, on=['ISO-alpha3 Code', 'Year'])

In [88]:
pd_final['GDP per capita'] = pd_final['Log GDP per capita'].apply(math.exp)

In [89]:
pd_final['GDP per capita_delta_abs'] = pd_final.groupby('Country name')['GDP per capita'].diff()
pd_final['GDP per capita_delta_rel'] = pd_final['GDP per capita_delta_abs'] / pd_final['GDP per capita'].shift(1)
pd_final = pd_final.dropna()
pd_final = pd_final.reset_index().drop('index', axis = 1)
pd_final['sentiment'] = pd.cut(pd_final['GDP per capita_delta_rel'], bins = [-99, 0, 0.03, 10], labels =['negative','neutral', 'positive'])

In [91]:
pd_final.to_pickle('./Data/Throughput/Speeches_processed_Happiness.pickle')


In [92]:
pd_final['sentence'] = preprocess(pd_final['Speech'], ['prep_lower'])
pd_final['sentence'] = pd_final['sentence'].apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())
pd_final[['sentence', 'sentiment']].to_csv('./Data/Throughput/Speeches_BERT.csv', index=False)